In [1]:
!pip install datasets scikit-learn python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from huggingface_hub import login
from dotenv import dotenv_values

login(token=dotenv_values('.env')['HUGGING_FACE_TOKEN'])

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\hakatashi\.cache\huggingface\token
Login successful


In [3]:
from datasets import load_dataset

dataset = load_dataset("hakatashi/hakatashi-pixiv-bookmark-deepdanbooru-private")

In [4]:
dataset

DatasetDict({
    test: Dataset({
        features: ['key', 'tag_probs', 'class'],
        num_rows: 59707
    })
    train: Dataset({
        features: ['key', 'tag_probs', 'class'],
        num_rows: 179121
    })
    validation: Dataset({
        features: ['key', 'tag_probs', 'class'],
        num_rows: 59708
    })
})

In [5]:
dataset['test'].features

{'key': Value(dtype='string', id=None),
 'tag_probs': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'class': ClassLabel(names=['not_bookmarked', 'bookmarked_public', 'bookmarked_private'], id=None)}

In [6]:
np_dataset = dataset.with_format(type='numpy')

In [7]:
train_data = np_dataset['train'][:]
test_data = np_dataset['test'][:]
validation_data = np_dataset['validation'][:]

In [8]:
from sklearn import svm
'''
svc = svm.SVC(verbose=1, kernel='rbf')
svc.max_iter = 10
svc.fit(train_data['tag_probs'], train_data['class'])
'''

"\nsvc = svm.SVC(verbose=1, kernel='rbf')\nsvc.max_iter = 10\nsvc.fit(train_data['tag_probs'], train_data['class'])\n"

In [9]:
'''
svc.score(test_data['tag_probs'], test_data['class'])
'''

"\nsvc.score(test_data['tag_probs'], test_data['class'])\n"

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

svc = svm.SVC(verbose=1, max_iter=150)
parameters = {
    'kernel': ('rbf', 'poly', 'sigmoid'),
    'C': (0.01, 0.1, 1),
    'gamma': ('scale', 0.000001, 0.01),
}
gscv = GridSearchCV(svc, parameters, scoring='accuracy', cv=3, verbose=2)
gscv.fit(train_data['tag_probs'], train_data['class'])

In [19]:
gscv.best_params_

{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}

In [20]:
gscv.best_score_

0.660609308791264

In [21]:
gscv.best_estimator_.score(test_data['tag_probs'], test_data['class'])

0.6631550739444286

In [22]:
predicted_class = gscv.predict(test_data['tag_probs'])

In [23]:
len(list(filter(lambda tup: tup[0] == 2 and tup[1] != 2, zip(predicted_class, test_data['class'], test_data['key']))))

9997

In [24]:
from joblib import dump
dump(gscv.best_estimator_, 'sklearn-multiclass-svc.joblib')

['sklearn-multiclass-svc.joblib']

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion_matrix:')
print(confusion_matrix(test_data["class"], predicted_class))
print(f'accuracy_score: {accuracy_score(test_data["class"], predicted_class)}')
print(f'precision_score: {precision_score(test_data["class"], predicted_class, average="macro")}')
print(f'recall_score: {recall_score(test_data["class"], predicted_class, average="macro")}')
print(f'f1_score: {f1_score(test_data["class"], predicted_class, average="macro")}')

confusion_matrix:
[[37384  1580  1734]
 [ 4757  1499  8263]
 [ 3491   287   712]]
accuracy_score: 0.6631550739444286
precision_score: 0.44369049752316053
recall_score: 0.39346319082079145
f1_score: 0.37579630871215225
